In [1]:
from auxiliary.auxiliary import *

plt.rcParams.update({'font.size': 14})

imagespath = 'C:/Users/Acer/PycharmProjects/BAMTclone/BAMT/experiments2/images'

Проблема: при перезапуске ядра строится БС с другой структурой. Синтетический датасет остаётся тем же, это было проверено.

In [2]:
n = 2000
np.random.seed(42)
data = pd.DataFrame(data=np.random.beta(0.95, 0.5, size=n)*6-3, columns=['origin1'])
data['origin2'] = np.random.beta(0.4,1, size=n)*2
data['cubed'] = data['origin1']**3/50 + np.random.normal(scale=0.1, size=n)
data['negative_root3'] = -np.sign(data['origin2'])*np.power(np.abs(data['origin2']), 1/3) + np.random.normal(scale=0.1, size=n)
data['combined'] = data['cubed'] + data['negative_root3']

data.head(10)

,origin1,origin2,cubed,negative_root3,combined
0,2.119492,0.737207,0.174883,-0.794917,-0.620034
1,-2.625095,0.031537,-0.477368,-0.337130,-0.814498
2,-2.894855,0.009888,-0.505449,-0.261298,-0.766747
3,2.688882,0.292918,0.486818,-0.737814,-0.250996
4,1.990172,0.243591,0.188168,-0.630120,-0.441953
5,0.055618,1.007663,0.114813,-1.010407,-0.895594
6,1.978326,1.391204,0.189030,-1.095673,-0.906644
7,2.810375,0.138295,0.314106,-0.530062,-0.215957
8,1.026379,0.008179,-0.230460,-0.214845,-0.445304
9,-0.542534,0.038656,0.011687,-0.415791,-0.404104


In [3]:
qresult = construct_by_quantiles(data, None)
bn = qresult['bn']

#c=bn.plot2("visualization_results" , "before1.html")
c=bn.plot2("visualization_results" , "after1.html") # uncomment to build network after kernel restart and comment row above

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


Такие БС получаются в двух различных перезапусках ядра (и вряд ли они получатся в следующих запусках)

![](issue_demo_before.png)

![](issue_demo_after.png)

Если же вернуться к обычному построению БС с узлами-признаками, то здесь тоже возникает разница от перезапуска ядра

In [4]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')

p = bamt.preprocessors.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
discretized_data, est = p.apply(data)

2023-04-10 13:28:36,410 | INFO     | preprocessors.py-scan-0091 | No one column is discrete


In [7]:
bn = Nets.ContinuousBN()

bn.add_nodes(p.info)

bn.add_edges(discretized_data,  scoring_function=("K2",K2Score))

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [8]:
c=bn.plot("default_before.html")
#c=bn.plot("default_after.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


![](default_before.png)

![](default_after.png)